In [2]:
import cv2
import numpy as np
import threading
import speech_recognition as sr

# Face recognition code
def face_recognition():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    id = 0
    names = ['None', 'Saugat', 'Sabira', 'Noman', 'Gagan', 'Daniel']
    
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)
    
    minW = 0.1 * cam.get(3)
    minH = 0.1 * cam.get(4)
    
    while True:
        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(int(minW), int(minH)),
        )
        
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
            
            if confidence < 100:
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
            
            cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
        
        cv2.imshow('camera', img)
        
        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break
    
    cam.release()
    cv2.destroyAllWindows()

# Speech to text code
def speech_to_text():
    r = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Talk")
        audio_text = r.listen(source, timeout=10)
        print("Time over, thanks")
        
        try:
            text = r.recognize_google(audio_text)
            print("Text: " + text)
        except:
            print("Sorry, I did not get that")

# Create threads for face recognition and speech to text
face_thread = threading.Thread(target=face_recognition)
speech_thread = threading.Thread(target=speech_to_text)

# Start both threads
face_thread.start()
speech_thread.start()

# Wait for both threads to finish
face_thread.join()
speech_thread.join()

Talk
